# Notebook 04: Reporte Final - Política de Inventario 2024
## Objetivo: Combinar el pronóstico base (GBR) con el cálculo de riesgo (GPD por producto) para generar la política de inventario final.

## Metodología:

- Cargar el pronóstico base de 2024 (data/processed/predicciones_demanda_2024.csv).

- Cargar el mapeo de stock de seguridad por producto (data/processed/stock_seguridad_por_producto.csv).

- Unir (merge) las dos tablas.

- Calcular la columna final: stock_total_recomendado = prediccion_ventas + stock_de_seguridad.

- Guardar el reporte final.

## 0. Configuración e Importación

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

## 1. Cargar Artefactos del Modelo
Cargamos las dos salidas de nuestros pipelines anteriores.

In [ ]:
# 1. Cargar el pronóstico base (12,000 filas = 1000 prod * 12 meses)
path_pronostico = '../data/processed/predicciones_demanda_2024.csv'
df_pronostico = pd.read_csv(path_pronostico)

print(f"Pronóstico base cargado: {len(df_pronostico)} filas.")
print(df_pronostico.head())

# 2. Cargar el stock de seguridad (1,000 filas = 1000 prod * 1 valor)
path_riesgo = '../data/processed/stock_seguridad_por_producto.csv'
df_riesgo = pd.read_csv(path_riesgo)

print(f"\nStock de seguridad cargado: {len(df_riesgo)} filas.")
print(df_riesgo.head())

## 2. Unir Pronóstico y Riesgo

Usamos un merge para asignar a cada una de las 12,000 filas de pronóstico el stock de seguridad que le corresponde a su prod_id.

In [ ]:
# Unir (merge) los dos dataframes usando 'prod_id' como llave
df_final = pd.merge(
    df_pronostico,
    df_riesgo,
    on='prod_id',
    how='left'
)

# Llenar con 0 si algún producto no tuvo cálculo de riesgo (ej. productos nuevos)
df_final['stock_de_seguridad'] = df_final['stock_de_seguridad'].fillna(0)

print("DataFrames unidos:")
print(df_final.head())

## 3. Calcular Política de Inventario Final

Aquí aplicamos la fórmula de negocio:

Stock Total a Planificar = prediccion_ventas + stock_de_seguridad

In [ ]:
# Aplicar la fórmula final
df_final['stock_total_recomendado'] = df_final['prediccion_ventas'] + df_final['stock_de_seguridad']

# Opcional: Redondear la predicción base
df_final['prediccion_ventas'] = df_final['prediccion_ventas'].round(0)

# Reordenar columnas para claridad
columnas_finales = [
    'fecha', 
    'prod_id', 
    'prediccion_ventas',      # Pronóstico base (media)
    'stock_de_seguridad',     # Riesgo (calculado por GPD)
    'stock_total_recomendado' # Decisión final
]

df_final = df_final[columnas_finales]

print("Política de inventario final generada:")
print(df_final.head())

## 4. Guardar Reporte Final

Este es el archivo CSV que se entrega al negocio.

In [ ]:
# Guardar el reporte final
path_salida_final = '../data/processed/politica_inventario_final_2024.csv'

df_final.to_csv(path_salida_final, index=False)

print(f"Reporte final guardado en: {path_salida_final}")

# Ver un ejemplo de un producto
print("\n--- Ejemplo para Producto 0 ---")
print(df_final[df_final['prod_id'] == 0])